# Main config file builder

*March 2018, Katherine McEwan (kmcewan@nikhef.nl), Erik Hogenbirk (ehogenbi@nikhef.nl, hogenbirk91@gmail.com)*

This script is intended to output configuration files for kodiaq, which is controlled by the daqcontrol. Find repository [here](https://github.com/ErikHogenbirk/XAMS). Adapted from the run8 version.

### Imports, settings, functions

In [167]:
# Imports
import json
import time
import os
from copy import deepcopy

In [168]:
# Configuration of the configuration maker
output_dir = '/home/xams/xams/daqcontrol/ini/'
filename = 'default_config.ini'

In [169]:
def set_register(cf, reg, val, comment="", verbose=False):
    '''This sets one register value of a config dict.'''
    # This is a list of dicts
    reg_list = cf['registers']
    new_reg_list = []
    reg_found = False
    for reg_dict in reg_list:
        # Find the dict with that register
        if reg_dict['register'] == reg:
            # If we found it, let's change it
            reg_dict['value'] = val
            if verbose: print("Just set register %s (%s) with value %s" % (reg_dict['register'],
                                                                          reg_dict['comment'], reg_dict['value']))
            # Keep track of if we found it
            reg_found = True
            new_reg_list.append(reg_dict)
        else:
            # If it doesn't match, we must just copy it
            new_reg_list.append(reg_dict)
    if not reg_found:
        # This is a NEW register, so we must build it
        print("Register %s not found in config, will append it now..." % reg)
        reg_dict = {
            "comment" : comment,
            "register": reg,
            "board" : "-1",
            "value" : val
        }
        new_reg_list.append(reg_dict)
    cf['registers'] = new_reg_list
    return cf
        

In [170]:
def save_config(cf, filename, output_dir=output_dir):
    config_path = os.path.join(output_dir, filename)
    with open(config_path, 'w') as outfile:
        json.dump(cf, outfile, indent = 4)
    print('Output written to %s' % config_path)

### Default

In [171]:
cf_default = {
    "data_folder" : "/home/xams/lena/run9/raw/data",
    "lena_pax_config_path" : "/data/xenon/xams/run9/pax_ini/_XAMSbase_light.ini",
    "zle" : False,
    "super_zle":False,
    "inverted_channels" : [],
    "links": [{
        "type": "V2718",
        "reader": 0,
        "crate": 0,
        "link": 0
    }],

    "boards": [{
        "crate": 0,
        "serial": "077",
        "reader": 0,
        "type": "V1724",
        "vme_address": "Whatever?",
        "link": 0
    }],
    # blt_size {int} : block transfer size in bytes (decimal).
    # V1724 standard: 524288
    # V1730 maximum: 2097152?
    "blt_size": 524288,
    # run_start {int} : 0 - board internal (run start via register), 
    # 1 - s-in (for synchronized boards)
    "run_start": 0,
    "baseline_mode": 0,
    # write_mode {int}: 0-no writing, 1-to file, 2-mongodb
    "write_mode": 2,

    "mongo": {
        "write_concern": 0,
        "min_insert_size": 1,
        "database": "xamsdata0",
        "address": "mongodb://localhost:27017",
        "collection": "onzin"
    },
    "processing_readout_threshold": 0,
    "processing_num_threads": 8,
    "processing_mode": 3,
    "occurrence_integral": 0,

    "compression": 1,
    
    "registers": [
        {
            "comment": "Reset software (all registers)",
            "register": "EF24",
            "board": "-1",
            "value": "1"
        },
        {
            "comment": "BLT event number",
            "register": "EF1C",
            "board": "-1",
            "value": "1"
            # Maximum number of events loaded into each block.
            # 40hex = 64dec
        },
        {
            "comment": "Enable BERR (error messages)",
            "register": "EF00",
            "board": "-1",
            "value": "10"
        },
        {
            "comment": "Aquisition control register",
            "register": "8100",
            "board": "-1",
            "value": "5" 
            #5: s-in control, 0: board control
        },
        {
            "comment": "Channel calibration",
            "register": "809C",
            "board": "-1",
            "value": "1"
            # enables channel calibration
        },
        {
            "comment": "Channel enable mask",
            "register": "8120",
            "board": "-1",
            "value": "09"
            # set to 01 to enable channel 0 only
            # set to 09 to enable channel 0 and 3
        },
        {
            "comment": "Measurement time after trigger",
            "register": "8114",
            "board": "-1",
            "value": "1400"
            # 20hex = 32dec = 320 samples = 640 ns
            # 1000 = 81 920 ns
        },
        {
            "comment": "Trigger source enable mask",
            "register": "810C",
            "board": "-1",
            "value": "C1F00003"
            #"value": "C1F00003" 120 ns
            # First digit: C for enable external + software trig
            # Second: coincidence level
            # Last one (?): channels enable
            # Set to "C0000001" for only trigger 0 and 1
        },
        {
            "comment": "Buffer organisation",
            "register": "800C",
            "board": "-1",
            "value": "6"
        },
        {
            "comment": "Interrupt status ID",
            "register": "EF14",
            "board": "-1",
            "value": "55AA"
            # Interrupt
        },
        {
            "comment": "Interrupt event number",
            "register": "EF18",
            "board": "-1",
            "value": "1"
            # Interrupt when 1 event stored
        },
        {
            "comment": "Channel 0 gain",
            "register": "1028",
            "board": "-1",
            "value": "0"
            # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
            # ONLY V1730
        },
        {
            "comment": "Channel 3 gain",
            "register": "1328",
            "board": "-1",
            "value": "0"
            # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
            # ONLY V1730
        },
        {
            "comment": "Trigger level channel 0",
            "register": "1080",
            "board": "-1",
            "value": "1FBE"
            # 1FF4 (hex) = 8180 (dec)
            # 1FEA (hex) = 8170 (dec)
            # 1FE0 (hex) = 8160 (dec)
            # 1FD6 (hex) = 8150 (dec)
            # 1F40 (hex) = 8000 (dec)
            # 1E78 (hex) = 7800 (dec)
        },
        {
            "comment": "Trigger level channel 3",
            "register": "1380",
            "board": "-1",
            "value": "2004"
        },
        {
            "comment": "Trigger polarity",
            "register": "8000",
            "board": "-1",
            "value": "40"
            # Set to 40 for negative, 00 for positive
        }
    ],
}


In [172]:
save_config(cf_default, 'default_config.ini')

Output written to /home/xams/xams/daqcontrol/ini/default_config.ini


## Other configuration

### Default ZLE

In [173]:
cf_default_zle = deepcopy(cf_default)
cf_default_zle['zle'] = True
save_config(cf_default_zle, 'default_zle.ini')

Output written to /home/xams/xams/daqcontrol/ini/default_zle.ini


### Gain calibration

Settings for PMT calibration. Triggers on a NIM pulse fed to channel 4. Short waveforms (1.28 us) with the PMT pulses that around 700 ns.

In [174]:
cf_PMT12_gain = deepcopy(cf_default)
cf_PMT12_gain['data_folder'] = "/home/xams/lena/run9/raw/gaincalibration"
cf_PMT12_gain["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_PMT12_gain.ini"

cf_PMT12_gain = set_register(cf_PMT12_gain, "8120", "19") # Chan enable
cf_PMT12_gain = set_register(cf_PMT12_gain, "810C", "C0000004") # Trigger source
cf_PMT12_gain = set_register(cf_PMT12_gain, "8114", "20")       # Time after trigger
cf_PMT12_gain = set_register(cf_PMT12_gain, "EF1C", "40")
cf_PMT12_gain = set_register(cf_PMT12_gain, "800C", "A")        # Buffer organization
cf_PMT12_gain = set_register(cf_PMT12_gain, "1028", "1")        # Channel 0 gain
cf_PMT12_gain = set_register(cf_PMT12_gain, "1328", "1")        # Channel 3 gain
cf_PMT12_gain = set_register(cf_PMT12_gain, "1428", "0")        # Channel 4 gain
cf_PMT12_gain = set_register(cf_PMT12_gain, "1080", "0000")     # Trigger level channel 0
cf_PMT12_gain = set_register(cf_PMT12_gain, "1380", "0000")     # Trigger level channel 3
cf_PMT12_gain = set_register(cf_PMT12_gain, "1480", "1770")     # Trigger level channel 4 (at 6000 counts, for NIM pulse)
cf_PMT12_gain = set_register(cf_PMT12_gain, "8020", "40")       # Event window size (40hex = 64dec = 
                                                                #                 640 samples = 1280 ns)
save_config(cf_PMT12_gain, 'PMT12_gain.ini')

Register 1428 not found in config, will append it now...
Register 1480 not found in config, will append it now...
Register 8020 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/PMT12_gain.ini


### Baseline

Same general configuration, but with the exception that you can trigger on external trigger only. Useful for noise, baseline, on data rate test.

In [175]:
cf_baseline = deepcopy(cf_default)
cf_baseline['data_folder'] = "/home/xams/lena/run9/raw/baseline"
cf_baseline["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_baseline.ini"
cf_baseline = set_register(cf_baseline, "810C", "C0000004")  # Trigger source: use ONLY external (or channel 4 0.5V pulse)
cf_baseline = set_register(cf_baseline, "1480", "1770")  # Trigger level channel 4 (-0.25 volt)
cf_baseline = set_register(cf_baseline, "1428", "0")  # Gain channel 4

# Test setting 22/03
# Default settings: baseline 0, 809C 1, 8098 unset
cf_baseline['baseline_mode'] = 0
cf_baseline = set_register(cf_baseline, "809C", '1')
cf_baseline = set_register(cf_baseline, "8098", "1000")


save_config(cf_baseline, 'baseline.ini')

Register 1480 not found in config, will append it now...
Register 1428 not found in config, will append it now...
Register 8098 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/baseline.ini


### Default_zle_long

In [176]:
cf_rn = deepcopy(cf_default)

cf_rn = set_register(cf_rn, "8114", "1900")     # Time after trigger
cf_rn = set_register(cf_rn, "800C", "5")        # Buffer organization
cf_rn = set_register(cf_rn, "8020", "2800")     # Custom window
cf_rn['zle'] = True
cf_rn['blt_size'] = 2097152


save_config(cf_rn, 'default_zle_long.ini')

Register 8020 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/default_zle_long.ini


### S1 only

In [177]:
cf_s1 = deepcopy(cf_default)
cf_s1['zle'] = True
cf_s1['data_folder'] = '/home/xams/lena/run9/raw/s1_only'
cf_s1['lena_pax_config_path'] = '/data/xenon/xams/run9/pax_ini/_XAMSbase_light.ini'
cf_s1 = set_register(cf_s1, "8114", "40")       # Time after trigger
                                                # 100hex = 256dec = 2560 samples = 5120 ns
cf_s1 = set_register(cf_s1, "800C", "A")        # Buffer organization
                                                # A = 10 us
cf_s1 = set_register(cf_s1, "8020", "80")       # Event window size (40hex = 64dec = 
                                                # 640 samples = 1280 ns)   
save_config(cf_s1, 's1_only.ini')

Register 8020 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/s1_only.ini


### Na22

In [178]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_Na22.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C2F00007") # Trigger source
cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4

save_config(cf_bla, 'Na22_zle.ini')

Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/Na22_zle.ini


### EJ passive

In [179]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_EJ.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
# cf_bla = set_register(cf_bla, "810C", "C2F00007") # Trigger source
# cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4
cf_bla = set_register(cf_bla, "1080", "1f86")
cf_bla = set_register(cf_bla, "1380", "1fc7")


save_config(cf_bla, 'EJ_passive.ini')

Output written to /home/xams/xams/daqcontrol/ini/EJ_passive.ini


### EJ coincidence

In [180]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_EJ.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C1F00006") # Trigger source
# cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4
# cf_bla = set_register(cf_bla, "1080", "1faf")
cf_bla = set_register(cf_bla, "1380", "203a") # trigger level channel 3
cf_bla = set_register(cf_bla, "1480", "1ffa")


save_config(cf_bla, 'EJ_coincidence_PMT1.ini')

Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/EJ_coincidence_PMT1.ini


### EJ or TPC

In [181]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_EJ.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C1F00007") # Trigger source
cf_bla = set_register(cf_bla, "1080", "1f86") # Trigger level channel 0
cf_bla = set_register(cf_bla, "1380", "203a")
cf_bla = set_register(cf_bla, "1480", "1ffa")


save_config(cf_bla, 'EJ_coincidence_PMT1_or_TPC.ini')

Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/EJ_coincidence_PMT1_or_TPC.ini


### EJ coincidence short

In [182]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_EJ.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C1F00006") # Trigger source
# cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4
# cf_bla = set_register(cf_bla, "1080", "1faf")
cf_bla = set_register(cf_bla, "1380", "203a")
cf_bla = set_register(cf_bla, "1480", "1ffa")
cf_bla = set_register(cf_bla, "8114", "96")     # Time after trigger
cf_bla = set_register(cf_bla, "800C", "A")        # Buffer organization

save_config(cf_bla, 'EJ_coincidence_PMT1_short.ini')

Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/EJ_coincidence_PMT1_short.ini


### EJ triggered short

In [183]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_EJ.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C0F00004") # Trigger source
# cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4
# cf_bla = set_register(cf_bla, "1080", "1faf")
# cf_bla = set_register(cf_bla, "1380", "203a")
cf_bla = set_register(cf_bla, "1480", "1ffa")
cf_bla = set_register(cf_bla, "8114", "96")     # Time after trigger
cf_bla = set_register(cf_bla, "800C", "A")        # Buffer organization

save_config(cf_bla, 'EJ_triggered_short.ini')

Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/EJ_triggered_short.ini


### Na22_long

In [184]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_Na22.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C2F00007") # Trigger source
cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4

cf_bla = set_register(cf_bla, "8114", "1900")     # Time after trigger
cf_bla = set_register(cf_bla, "800C", "5")        # Buffer organization
cf_bla = set_register(cf_bla, "8020", "2800")     # Custom window
cf_bla['blt_size'] = 2097152

save_config(cf_bla, 'Na22_zle_long.ini')

Register 1480 not found in config, will append it now...
Register 8020 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/Na22_zle_long.ini


### Na22_superlong

In [185]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_Na22.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C2F00007") # Trigger source
cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4

cf_bla = set_register(cf_bla, "8114", "2600")     # Time after trigger
cf_bla = set_register(cf_bla, "800C", "4")        # Buffer organization
cf_bla = set_register(cf_bla, "8020", "4000")     # Custom window
cf_bla['blt_size'] = 2097152

save_config(cf_bla, 'Na22_zle_superlong.ini')

Register 1480 not found in config, will append it now...
Register 8020 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/Na22_zle_superlong.ini


In [186]:
int('2800', 16)

10240

In [187]:
int('4000', 16) * 2 * 0.01

327.68

### Na22_zerofield

In [188]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/data"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_Na22.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C2F00007") # Trigger source
cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4

cf_bla = set_register(cf_bla, "8114", "96")     # Time after trigger
cf_bla = set_register(cf_bla, "800C", "A")        # Buffer organization

save_config(cf_bla, 'Na22_zerofield.ini')

Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/Na22_zerofield.ini


### NaI_only

In [189]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = False

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/NaI_only"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_scin.ini"

cf_bla = set_register(cf_bla, "8114", "96")       # Time after trigger
cf_bla = set_register(cf_bla, "800C", "A")        # Buffer organization
cf_bla = set_register(cf_bla, "8020", "15E")       # Event window size (40hex = 1280ns)
cf_bla = set_register(cf_bla, "8120", "01")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C0000001") # Trigger source
cf_bla = set_register(cf_bla, "1080", "1FA4")     # Trigger level channel 0
cf_bla = set_register(cf_bla, "1180", "0000")     # Trigger level channel 1


save_config(cf_bla, 'NaI_only.ini')

Register 8020 not found in config, will append it now...
Register 1180 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/NaI_only.ini


### EJ_only

In [190]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = False

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/NaI_only"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_scin_ch4.ini"

cf_bla = set_register(cf_bla, "8114", "96")       # Time after trigger
cf_bla = set_register(cf_bla, "800C", "A")        # Buffer organization
cf_bla = set_register(cf_bla, "8020", "15E")       # Event window size (40hex = 1280ns)
cf_bla = set_register(cf_bla, "8120", "10")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C0000004") # Trigger source
cf_bla = set_register(cf_bla, "1480", "1ffa")     # Trigger level channel 4

save_config(cf_bla, 'EJ_only.ini')

Register 8020 not found in config, will append it now...
Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/EJ_only.ini


### NaI_only + Discriminator

In [191]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = False

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/NaI_only"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_scin_coincidence.ini"

cf_bla = set_register(cf_bla, "8114", "96")       # Time after trigger
cf_bla = set_register(cf_bla, "800C", "A")        # Buffer organization
cf_bla = set_register(cf_bla, "8020", "15E")       # Event window size (40hex = 1280ns)
cf_bla = set_register(cf_bla, "8120", "09")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C1F00003") # Trigger source
cf_bla = set_register(cf_bla, "1080", "1FEA")     # Trigger level channel 0
cf_bla = set_register(cf_bla, "1180", "0000")     # Trigger level channel 1
cf_bla = set_register(cf_bla, "1280", "0000")     # Trigger level channel 2
cf_bla = set_register(cf_bla, "1380", "1900")     # Trigger level channel 3


save_config(cf_bla, 'NaI_only.ini')

#Some stuff was changed here but w/e

Register 8020 not found in config, will append it now...
Register 1180 not found in config, will append it now...
Register 1280 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/NaI_only.ini


### NaI_coincidence

In [192]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = False

cf_bla['data_folder'] = "/home/xams/lena/run9/raw/NaI_only"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_scin_coincidence.ini"

cf_bla = set_register(cf_bla, "8114", "60")       # Time after trigger
cf_bla = set_register(cf_bla, "800C", "A")        # Buffer organization
cf_bla = set_register(cf_bla, "8020", "C0")       # Event window size (40hex = 1280ns)
cf_bla = set_register(cf_bla, "8120", "B")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C1F00003") # Trigger source
cf_bla = set_register(cf_bla, "1080", "1FEA")     # Trigger level channel 0
cf_bla = set_register(cf_bla, "1180", "0000")     # Trigger level channel 1
cf_bla = set_register(cf_bla, "1280", "0000")     # Trigger level channel 2
cf_bla = set_register(cf_bla, "1380", "1FA4")     # Trigger level channel 3


save_config(cf_bla, 'NaI_coincidence.ini')

Register 8020 not found in config, will append it now...
Register 1180 not found in config, will append it now...
Register 1280 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/NaI_coincidence.ini


### SiPM

In [193]:
cf_PMT12_gain = deepcopy(cf_default)
cf_PMT12_gain['data_folder'] = "/data/xenon/xams/run9/raw/sipm"
cf_PMT12_gain["lena_pax_config_path"] = "/data/xenon/xams/run9/pax_ini/XAMS_raw_to_processed_sipm.ini"
cf_PMT12_gain["inverted_channels"] = [0]

cf_PMT12_gain = set_register(cf_PMT12_gain, "8120", "11") # Chan enable
cf_PMT12_gain = set_register(cf_PMT12_gain, "810C", "C0000004") # Trigger source channel 4
cf_PMT12_gain = set_register(cf_PMT12_gain, "8114", "20")       # Time after trigger
cf_PMT12_gain = set_register(cf_PMT12_gain, "800C", "A")        # Buffer organization
cf_PMT12_gain = set_register(cf_PMT12_gain, "1028", "1")        # Channel 0 gain smaller dynamic range = 1
cf_PMT12_gain = set_register(cf_PMT12_gain, "1428", "0")        # Channel 4 gain pulse generator
cf_PMT12_gain = set_register(cf_PMT12_gain, "1480", "1770")     # Trigger level channel 4 (at 6000 counts, for NIM pulse)
cf_PMT12_gain = set_register(cf_PMT12_gain, "8020", "40")       # Event window size (40hex = 64dec = 
                                                                #                 640 samples = 1280 ns)
save_config(cf_PMT12_gain, 'SiPM_gain.ini')

Register 1428 not found in config, will append it now...
Register 1480 not found in config, will append it now...
Register 8020 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/SiPM_gain.ini


## Auxilary

In [194]:
def get_trigger_level(channel, counts):
    print('%d counts = %.1f mV' % (counts, counts/16383*2000))
    if channel==0:
        level = 8193 - counts
    
    if channel ==3:
        level = 8258 - counts
        
    if channel == 4:
        level = 8268 - counts
        
    return hex(level)

In [195]:
def invert_trigger_level(channel, counts):
    counts = int(counts, 16)
    if channel==0:
        level = 8193 - counts
    
    if channel ==3:
        level = 8258 - counts
        
    if channel == 4:
        level = 8268 - counts
        
    return level

In [196]:
invert_trigger_level(0, '1FA4')

93

In [197]:
invert_trigger_level(3, '1ff0')

82

In [198]:
def get_window_size(hex_string):
    hex_string = str(hex_string)
    return int(hex_string, 16) * 10 * 2 * 1e-3

In [199]:
def get_trigger_delay(hex_string):
    hex_string = str(hex_string)
    print('Warning: uses ad-hoc guess of computation')
    return int(hex_string, 16) * 8 * 2 * 1e-3

In [200]:
get_trigger_level(0, 93)

93 counts = 11.4 mV


'0x1fa4'

In [201]:
get_trigger_level(4, 150)

150 counts = 18.3 mV


'0x1fb6'

In [202]:
get_trigger_level(3, 123)

123 counts = 15.0 mV


'0x1fc7'

In [203]:
get_trigger_level(3, 16)

16 counts = 2.0 mV


'0x2032'

In [204]:
get_trigger_level(3, 27)

27 counts = 3.3 mV


'0x2027'

In [205]:
get_trigger_level(3, 8)

8 counts = 1.0 mV


'0x203a'

In [206]:
int('15E', 16)

350

In [207]:
int('1fc8', 16)

8136

In [208]:
get_window_size('15e')

7.0

In [209]:
50*1.6022e-19*5e8 * 2.3e6 *1e3

9.21265

In [210]:
print((8189-int('1fbe', 16))/16000*2000)
print((8257-int('2004', 16))/16000*2000)

7.875
7.625


## Other

In [211]:
import datetime
now = datetime.datetime.now()
print('Done dumping config files! Last update: %s'
      % (now.strftime("%Y-%m-%d %H:%M")))

Done dumping config files! Last update: 2018-12-06 15:09
